# Team Give Me Pizza

## Pizza Project

* Adam Sayre
* Chenxi Lu
* Lina Sheremet
* Scott Robertson

In [26]:
import nltk
nltk.download('stopwords') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# Import a bunch of libraries.
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# General libraries.
import re

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# for preprocessors
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

from scipy.sparse import coo_matrix, hstack
from nltk.tokenize import sent_tokenize, word_tokenize


from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import AdaBoostClassifier 

import sys, os
import torch
import torch.nn as nn
import torch.utils.data
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score
from sklearn import metrics

%load_ext autoreload
%autoreload 2
%matplotlib inline
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
import shutil

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
# Set the randomizer seed so results are the same each time.
np.random.seed(0)

input_data = pd.read_json('/content/drive/My Drive/train.json')
input_data.shape

input_data = input_data.sample(frac=1)

dev_data = input_data[0:1347]
train_data = input_data[1347:]

print('Train data shape: ', train_data.shape)
print('Dev data shape:', dev_data.shape)

train_labels = train_data['requester_received_pizza']
dev_labels = dev_data['requester_received_pizza']

print('Train labels shape: ', train_labels.shape)
print('Dev labels shape:', dev_labels.shape)


(4040, 32)

Train data shape:  (2693, 32)
Dev data shape: (1347, 32)
Train labels shape:  (2693,)
Dev labels shape: (1347,)


In [30]:
#################################################
# One hot encoding of True / False
boo = list(train_data.index)
onehot_col_train = []
for i in range(len(boo)):
  x = train_data[i:i+1]
  if x["requester_received_pizza"][boo[i]] == True:
    onehot_col_train.append(1)
  else:
    onehot_col_train.append(0)

boo2 = list(dev_data.index)
onehot_col_dev = []
for i in range(len(boo2)):
  x = dev_data[i:i+1]
  if x["requester_received_pizza"][boo2[i]] == True:
    onehot_col_dev.append(1)
  else:
    onehot_col_dev.append(0)
#################################################

# Add encoded True / False
train_data["encoding"] = onehot_col_train
# Add encoded True / False
dev_data["encoding"] = onehot_col_dev


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [31]:
# Download the pretrained weights for bert base. 
!wget -nv --show-progress -O data/uncased_L-12_H-768_A-12.zip \
        https://storage.googleapis.com/bert_models/2018_10_18/uncased_L-12_H-768_A-12.zip
!wget -nv --show-progress  -O data/cased_L-12_H-768_A-12.zip \
        https://storage.googleapis.com/bert_models/2018_10_18/cased_L-12_H-768_A-12.zip
# unzip weights & conifg and remove the original zip
!unzip -d data/ data/cased_L-12_H-768_A-12.zip && rm data/cased_L-12_H-768_A-12.zip   
!unzip -d data/ data/uncased_L-12_H-768_A-12.zip && rm data/uncased_L-12_H-768_A-12.zip

data/uncased_L-12_H-768_A-12.zip: No such file or directory
data/cased_L-12_H-768_A-12.zip: No such file or directory
unzip:  cannot find or open data/cased_L-12_H-768_A-12.zip, data/cased_L-12_H-768_A-12.zip.zip or data/cased_L-12_H-768_A-12.zip.ZIP.
unzip:  cannot find or open data/uncased_L-12_H-768_A-12.zip, data/uncased_L-12_H-768_A-12.zip.zip or data/uncased_L-12_H-768_A-12.zip.ZIP.


In [32]:
pip install torchvision

In [33]:
# Let's activate CUDA for GPU based operations
device=torch.device('cuda')

In [34]:
# In bert we need all inputs to have the same length, we will use the first 220 characters. 
MAX_SEQUENCE_LENGTH = 220
SEED = 1234
# We shall run a single epoch (ie. one pass over the data)
EPOCHS = 1
# PATH = '.'#'/root/v2/week06/hw' # /root/v2/week06/hw"
# DATA_DIR = os.path.join(PATH, "data")
# WORK_DIR = os.path.join(PATH, "workingdir")

# Validation and training sizes are here. 
train_size= 1000
valid_size= 500

In [35]:
# os.listdir(DATA_DIR)

In [36]:
%%capture
!pip install transformers
from transformers import BertModel, BertConfig, BertTokenizer, BertForSequenceClassification
from transformers import AdamW as BertAdam

In [37]:
%%capture
bert_config = BertConfig()

In [38]:
def convert_lines(example, max_seq_length,tokenizer):
    max_seq_length -=2
    all_tokens = []
    longer = 0
    for text in tqdm_notebook(example):
        tokens_a = tokenizer.tokenize(text)
        if len(tokens_a)>max_seq_length:
            tokens_a = tokens_a[:max_seq_length]
            longer += 1
        one_token = tokenizer.convert_tokens_to_ids(["[CLS]"]+tokens_a+["[SEP]"])+[0] * (max_seq_length - len(tokens_a))
        all_tokens.append(one_token)
    print(longer)
    return np.array(all_tokens)

In [39]:
# %%time
# tokenizer = BertTokenizer.from_pretrained(BERT_MODEL_PATH, cache_dir=None,do_lower_case=True)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_all = train_data.sample(train_size+valid_size,random_state=SEED)
print('loaded %d records' % len(train_all))

# Make sure all comment_text values are strings
train_all['request_text_edit_aware'] = train_all['request_text_edit_aware'].astype(str) 

sequences = convert_lines(train_all["request_text_edit_aware"].fillna("DUMMY_VALUE"),MAX_SEQUENCE_LENGTH,tokenizer)
train_all=train_all.fillna(0)

loaded 1500 records


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """



101


In [40]:
train_all[["request_text_edit_aware", 'encoding']].head()

,request_text_edit_aware,encoding
611,"We just moved to a completely unfamiliar area,...",1
1387,My alarm did not go off before work this after...,0
3271,"hey, my sister just turned 15 and her birthday...",0
459,"Hello fellow pizza enthusiasts, it's been a ro...",1
3297,"I'm a student, and my fridge is empty. I would...",0


In [49]:
text = 'Hi, I would like as many pizzas as possssssible please.'
tokens = tokenizer.tokenize(text)
' '.join(tokens)

'hi , i would like as many pizza ##s as po ##ss ##ss ##ssi ##ble please .'

In [50]:
tokens = ["[CLS]"] + tokens + ["[SEP]"]
input_ids = tokenizer.convert_tokens_to_ids(tokens)
' '.join(map(str, input_ids))

'101 7632 1010 1045 2052 2066 2004 2116 10733 2015 2004 13433 4757 4757 18719 3468 3531 1012 102'

In [51]:
# put input on gpu and make prediction
bert = BertForSequenceClassification.from_pretrained('bert-base-uncased').cuda()
# bert = BertModel.from_pretrained(WORK_DIR).cuda()
bert_output = bert(torch.tensor([input_ids])).cuda()

print('Sentence tokens {}'.format(tokens))
print('Number of tokens {}'.format(len(tokens)))
print('Tensor shapes : {}'.format([b.cpu().detach().numpy().shape for b in bert_output[0]]))
print('Number of torch tensors : {}'.format(len(bert_output[0])))

Sentence tokens ['[CLS]', 'hi', ',', 'i', 'would', 'like', 'as', 'many', 'pizza', '##s', 'as', 'po', '##ss', '##ss', '##ssi', '##ble', 'please', '.', '[SEP]']
Number of tokens 19
Tensor shapes : [(2,)]
Number of torch tensors : 1


In [52]:
train_all['encoding']=(train_all['encoding']>=0.5).astype(float)
# Training data - sentences
X = sequences[:train_size] 
# Target - the toxicity. 
y = train_all[['requester_received_pizza']].values[:train_size]
X_val = sequences[train_size:]                
y_val = train_all[['requester_received_pizza']].values[train_size:]

In [53]:
test_df=train_all.tail(valid_size).copy()
train_df=train_all.head(train_size)

In [54]:
# Training data creations
train_dataset = torch.utils.data.TensorDataset(torch.tensor(X,dtype=torch.long), torch.tensor(y,dtype=torch.float))

In [55]:
SEED = 1234
lr=2e-5
batch_size = 32
# batch_size=4
accumulation_steps = 1
# accumulation_steps = 8
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [56]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',cache_dir=None,num_labels=1)
model.zero_grad()
model = model.to(device)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]


In [1]:
train = train_dataset
tq = tqdm_notebook(range(EPOCHS))
num_train_optimization_steps = int(EPOCHS*len(train)/batch_size/accumulation_steps)

optimizer = BertAdam(optimizer_grouped_parameters, lr=lr)


lossf=None
loss_fn=nn.MSELoss()
scaler = torch.cuda.amp.GradScaler()
for epochz in tq:
    train_loader = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)
    avg_loss = 0
    avg_accuracy = 0
    tk0= tqdm_notebook(enumerate(train_loader), total=len(train_loader),leave=False)
    for i,(x_batch, y_batch) in tk0:
        optimizer.zero_grad()
        with torch.cuda.amp.autocast():
            y_pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
            loss = F.binary_cross_entropy_with_logits(y_pred,y_batch.to(device))
        scaler.scale(loss).backward()
        if (i+1) % accumulation_steps == 0:             # Wait for several backward steps
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        if lossf:
            lossf = 0.98*lossf+0.02*loss.item()
        else:
            lossf = loss.item()

        tk0.set_postfix(loss = lossf)
        avg_loss += loss.item() / len(train_loader)
        avg_accuracy += torch.mean(((torch.sigmoid(y_pred[:,0])>0.5) == (y_batch[:,0]>0.5).to(device)).to(torch.float) ).item()/len(train_loader)
tq.set_postfix(avg_loss=avg_loss,avg_accuracy=avg_accuracy)



NameError: ignored

In [ ]:
torch.save(model.state_dict(), 'bert_pytorch.bin')

In [ ]:
%%capture 
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=1).cuda()
model.load_state_dict(torch.load('bert_pytorch.bin'))
model.to(device)
for param in model.parameters():
    param.requires_grad=False
model.eval()  # print the model ---> you can remove 
val_batch_size = 128
valid_preds = np.zeros((len(X_val)))
valid = torch.utils.data.TensorDataset(torch.tensor(X_val,dtype=torch.long))
valid_loader = torch.utils.data.DataLoader(valid, batch_size=val_batch_size, shuffle=False)
tk0 = tqdm_notebook(valid_loader)
for i,(x_batch,)  in enumerate(tk0):
    pred = model(x_batch.to(device), attention_mask=(x_batch>0).to(device), labels=None)[0]
    valid_preds[i*val_batch_size:(i+1)*val_batch_size]=pred[:,0].detach().cpu().squeeze().numpy()
    if i % 20 == 0:
        print("batch: ", i, " of ", len(valid_loader))

In [ ]:
MODEL_NAME = 'model'
test_df[MODEL_NAME]=torch.sigmoid(torch.tensor(valid_preds)).numpy()
encode_column = 'encoding'
bias_metrics_df = compute_bias_metrics_for_model(test_df, identity_columns, MODEL_NAME, 'encoding')
bias_metrics_df
get_final_metric(bias_metrics_df, calculate_overall_auc(test_df, MODEL_NAME))